# Beskrivelse
Her gives det det samme kode som i Eksemple_CNN.ipynb. Modelen der kunne når et test accuracy på 95-96%. Her skal vi undersøg om vi kan lave modellen bedre, eller dårligere, og se hvordan at ændre de forskellige parametere har effekt på resultatet. Først starter vi med at kopiere hjælpe kode filerne:

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    !cp -r "/content/drive/MyDrive/Colab Notebooks/ML-Camp-2025_CNN_Dev/ConvNets/utils" .
except ImportError:
    print("Not in colab. Skipping copying the utilities files.")

Derefter importerer vi pakker og overfører datasætet.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
from utils.train import train, plot_training_logs
from utils.test import test
from utils.options import Hyperparameters, name_generator

# Tjekke om der er GPU ellers bruge CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Denne transform funktion gives til datasættet for at billederne kommer ud i den rigtig format, som er matricer med værdier mellem 0 og 1.
# Vi normalisere pixlerne fra [0, 255] til [0, 1] fordi mest ML algoritmer er bygget til at arbejde bedst med normaliseret data.
def image_transform(img):
    return torchvision.transforms.ToTensor()(img).unsqueeze(0)

# Overfører CIFAR10 træning og test datasætene fra pytorch
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=image_transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=image_transform)

# Splitter træning sætet til en træning og validering sæt.
# val_set_ratio bestemmer hvor meget af sættet bliver brugt til validering.
val_set_ratio = 0.1
train_set, val_set = random_split(train_set, [int(len(train_set)*(1-val_set_ratio)), int(len(train_set)*val_set_ratio)])

# Tjekker størrelsen af billederne og hvor mange klasser der er
print("Images shape:", train_set[0][0].shape)
print("Number of classes:", len(np.unique(test_set.targets)))

# Model / data parameter
num_classes = 10
input_shape = (1, 28, 28)

# Laver dataloadere der samler vores data i batches og shuffler dem hvis vi vil gerne
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

Køre den initial model igen så du kan sammenlign med den senere:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "base_model",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

## Opgaver: Undersøg forskellige modeller
Igennem disse opgaver vil du finde instruktioner i øverest, og så vil du finde steder hvor der står "fix" eller "NOTE:, som kan hjælpe dig med at kigge efter hvor du skal ændre i koden.

### Convolution lagene
Nu skal du ændre parameter i modellen og se hvis de har en effekt på performance (val_accuracy) og træning hastighed (ms/step). Første prøv at ændre antal af convolution lag. Den første convolution lag med dens relu aktivering er blevet fjernet. Fix koden og se hvad der sker med validering accuracy. **Husk at opdatere input størrelsen for conv2 og kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        #self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(input_channels, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 13, 13)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        #x = self.conv1(x)
        #x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "no_conv1",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Nu er den anden convolution lag fjernet istedet med dens relu aktivering. **Husk at opdatere kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        #self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 26, 26)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        #x = self.conv2(x)
        #x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecture.to(device)ne fra model_architecture.py
model = Net(
    name = "no_conv2",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Hvad hvis du sætter et ekstra convolution lag. Initialisere en ny Conv2D og sæt den efter MaxPooling2D. Du kan selv vælge parametrene af laget, og **husk at opdaterer input størrelse + kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # NOTE: Indsæt en ekstra convolution lag her. I kan kalde den self.conv_extra eller hvad som helst
        self.convExtra = nn.Conv2d(64, 96, 5, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(96, 128, 8, 8)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        # NOTE: Pass x igennem den ny convolution lag her
        x = self.convExtra(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "ekstra_conv",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Sammenlign hastighed og accuracy forskel mellem de tre sidste modeller og den basal model. Var de bedre eller dårligere? Hurtigere eller langsomere? Hvorfor tror du det?

### Max pooling lagene
Nu kigger vi på max pooling. Den ene max pooling lag er blevet fjernet fra modellen. Fix koden og se hvad der sker med hastighed og accuracy. **Husk at opdaterer kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 24, 24)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "no_max_pool",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Nu prøve at sætte et ekstre max pooling lag efter conv1 og dens aktivering. **Husk at opdaterer kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 5, 5)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        # NOTE: Lav en max_pool2d her
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "extra_max_pool",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Hvad hvis vi bruger større kerner? Prøv kerne størrelse på 4 istedet for 2 i de to max pooling lage. Sådan at billederne bliver mindre. **Husk at opdaterer kerne størrelse + stride for conv3**:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 1, 1)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 4)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 4)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "extra_max_pool4",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Sammenlign de sidste tre modeler der bruger forskellige max pooling lag med den initial model. Hvorfor tror du der er forskel i træning hastighed og accuracy?

### Aktivering funktionet
Aktivering funktionerne er brugt til at gør sikkert at modelen kan også lære mønstre på ulineart data, da en model uden aktivering kan kun finde lineart afhængigheder mellem dataen. Hvis man fjerner alle aktivering funktioner, så bliver det til linær aktiverin, da ingen aktivering er bare $f(x)=x$, som er linear aktiveringsfunktion.

Man kan også skift aktivering funktion til en anden ved at bruge et andet funktion end F.relu. Man kan finde resten af funktionerne [her](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions).

Træn en model der bruger kun linear aktivering (ingen aktivering), fjern ikke F.log_softmax, fordi den bruges til at lave netværkets output vektor til en sandsynlighedsvektor:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        # NOTE: Fjern de linjer som kører relu aktivering
        x = self.conv1(x)
        #x = F.relu(x)
        x = self.conv2(x)
        #x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        #x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "linear_model",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Der findes også andre ikke lineart funktioner, prøv sigmoid og tanh aktivering funktionerne:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        # NOTE: Skift F.relu med F.sigmoid eller F.tanh
        x = self.conv1(x)
        x = F.sigmoid(x)
        x = self.conv2(x)
        x = F.sigmoid(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.sigmoid(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "different_activation1",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        # NOTE: Skift F.relu med F.tanh eller F.sigmoid
        x = self.conv1(x)
        x = F.tanh(x)
        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.tanh(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "different_activation2",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Sammenlign de forskellige aktiveringsfunktioner. Skal man altid bruge et non-lineart aktiveringsfunktion? Får man hurtigere træning ved at bruge lineart aktivering istedet for ikke lineart aktivering? Er de andre aktivering funktioner så god som RELU?

## Opgave: Undersøg træning parameter
Det er ikke kun modellen der er vigtig for at få god resultater. Selve træning processen er også vigtigt.

Epochen bestemer hvor langt modellen skal træn, for mange epocher kan resulterer i overfitting, og for færre epocher resulterer i underfitting.

Batch størrelsen bestemmer hvor mange data punkter man bruge til at adjusterer vægtene med og hvor mange steps man tager per epoch som bestemmer hvor hurtigt vægtene ændres. Større batches resulterer i mere præcise vægt ændringer, men kan også resultarer i langsommere træning.

Der også findes mange forskellige optimering algoritmer der opdaterer vægtene.

### Antal Epocher
Nu skal du undersøg om de førrig modeller underfitter. Prøv at øve antal epocher og se hvordan accuracy ændres. Pas på med at gå alt for stort, da flere epocher tager mere tid til at træne prøv at ikke gå over 80:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 50,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "different_epochs",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Hint: Et måde at finde en god epoch mængde er at starte med et stort epoch tal, og så se hvornår validering accuracy starter med at ændrer sig for lidt mellem hvert epoch (fx. 0.005). Dette strategi kaldes for early stopping, og den bruges for at gøre sikker at modellen træner ikke mere end den har bruge for.

Hvad fandt du til at være en god mængde af epocher? Giver det mening at bruge det ekstra tid til at for den mængde øvede accuracy man får? Hvad hvis din model tog 10 minutter eller en time per epoch? 

### Andre optimering algoritmer
Lige nu bruges der SGD som optimering algoritm. Der andre optimizers i pytorch som kan ses [her](https://pytorch.org/docs/stable/optim.html#algorithms) prøv at bruge adam istedet for SGD, som er nyere og er den mest brugt inden for ML lige nu.

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag
        self.conv1 = nn.Conv2d(input_channels, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.conv3 = nn.Conv2d(64, 128, 12, 12)
        self.conv4 = nn.Conv2d(128, num_classes, 1, 1)

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.conv4(x)
        x = x.mean(dim=(2,3))
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.Adam
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "adam_optimizer",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

Var Adam bedre end SGD? Har Adam bruge for så mange Epochs som SGD?

Prøv at træne modellen for 50 epochs. Brug ML-flow for at se om Adam overfitter datasættet. Det kan man se ved at kigge på hvordan træning loss og validering loss udvikler sig under træning.

## Opgave: En Sidste Model
Nu har du prøvet mange forskellige modeller. Prøv at træne en model med de parametere du tror vil virke bedste og træn det. Når du tror at du har trænet den bedste model, så køre test kodeblokken og se hvor god din model kan detekterer hånd skrevet nummer:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_channels = 1, num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # NOTE: Initialiserer model lag (Husk at slette raise linjen når du færdig)
        raise NotImplementedError("Implementer Netværksarkitektur.")

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        # NOTE: Implementere forward pass (Husk at slette raise linjen når du færdig)
        raise NotImplementedError("Implementer forward pass")
        output = F.log_softmax(x, dim=1)
        return output
    
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, name, hyperparameters: dict = {}, input_shape = (1, 28, 28), num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()

        # Navngiv model
        self.name = name

        # Load Hyperparametre
        self.hyperparameters = hyperparameters

        # Vælg loss function
        self.criterion = nn.CrossEntropyLoss()
        setattr(self.hyperparameters, 'loss', self.criterion.__class__.__name__)

        # Initialiserer model lag (Husk at slette raise linjen når du færdig)
        self.input_shape = input_shape
        raise NotImplementedError("Implementer Netværksarkitektur.")
        

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = x.reshape([-1] + list(self.input_shape))
        # Implementere forward pass (Husk at slette raise linjen når du færdig)
        raise NotImplementedError("Implementer forward pass")
        output = F.log_softmax(x, dim=1)
        return output

# Sæt valgmuligheder
hyperparameters = Hyperparameters(
    lr = 0.005,
    epochs = 20,
    optimizer = optim.SGD,
)

# Hent model architecturene fra model_architecture.py
model = Net(
    name = "Final_model",
    hyperparameters=hyperparameters, 
    input_shape=input_shape, 
    num_classes=num_classes
).to(device)

# tilføj optimizer til model
model.optimizer = model.hyperparameters.optimizer(
    model.parameters(),
    lr=model.hyperparameters.lr,
    momentum=model.hyperparameters.momentum,
)
setattr(model.hyperparameters, 'optimizer', model.optimizer.__class__.__name__)

logs = train(train_loader, val_loader, model)

Fig, ax = plot_training_logs(logs)
Fig.show()

## VENT!
Husk at man burde kun bruge test datasætet en gang til sidste. Er du sikker at din nuværende model er den du vil gerne teste på?

In [ ]:
model_path = "saved_models/Final_model_best.pt"
model = torch.jit.load(model_path, map_location=device)
test(test_loader, model)

Hvor meget bedre er denne model end den initial model i Eksemple_CNN.ipynb?